In [6]:
using Statistics, StatsBase, JLD, Random
using LinearAlgebra, IntervalArithmetic

setprecision(BigFloat,div(196,2))
const T = Interval{BigFloat} # yeah high precision intervals baby
const p = T.((180//100,35//100))
const N = 10^4; # how many cool_intervals calls per sim
const K = 60 # how long to push forward for

include("../simulations/cool-intervals.jl")

cool_intervals (generic function with 1 method)

In [9]:
# start with the backward-invariant unstable manifold between w_0 and w_1
beta = (p[1]-sqrt(p[1]^2+4p[2]))/2; alpha = -beta/(1+beta)
x_ref_end = -alpha/beta; x_ref_start = 1-p[1]*x_ref_end
Q = [1 -beta; beta 1]/sqrt(1+beta*beta)
Is = Q[2,:]*alpha .+ sqrt(1+beta*beta)*T.([1,0])*[x_ref_start x_ref_end]

# spinup
for i = 1:10000
    Is, Q, c, cw = cool_intervals(Is,Q,p)
end

attractor_endpts = Array{Float64}(undef,2,2,N)
sigma_pts = Array{Float64}(undef,N)
sigma_wts = Array{Float64}(undef,N)
sigma_pushforward = Array{Float64}(undef,2,K,N)
for i = 1:N
    Is, Q, c, cw = cool_intervals(Is,Q,p)
    attractor_endpts[:,:,i] = Float64.(mid.(Q'*Is))
    sigma_pts[i] = c; sigma_wts[i] = cw
    if ~isnan(c)
        x = [0,c]
        for k = 1:K
            smap!(x,p)
            sigma_pushforward[:,k,i] = Float64.(mid.(x))
        end
    end
end

LoadError: InterruptException: